In [14]:
import numpy as np
import pandas as pd

In [16]:
experiment = '1a'

df = pd.read_csv(f'results/{experiment}/processed_results_{experiment}.csv', index_col='Unnamed: 0')
df

,models,accs,seeds,c11,c12,c21,c22
0,ngc,1.0,1,0.0,0.0,0.0,0.0
1,ngc,1.0,101,0.0,0.0,0.0,0.0
2,ngc,1.0,33,0.0,0.0,0.0,0.0
3,ngc,1.0,12,0.0,0.0,0.0,0.0
4,ngc,1.0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
145,nri,0.0,1,0.9,0.0,0.0,0.9
146,nri,0.5,101,0.9,0.0,0.0,0.9
147,nri,0.5,33,0.9,0.0,0.0,0.9
148,nri,0.5,12,0.9,0.0,0.0,0.9


In [20]:
df.groupby(by=['models', 'c11', 'c12', 'c21', 'c22']).mean()

accs  seeds
models c11 c12 c21 c22             
ngc    0.0 0.0 0.0 0.0  1.00   29.4
       0.1 0.0 0.0 0.1  0.50   29.4
       0.2 0.0 0.0 0.2  0.50   29.4
       0.3 0.0 0.0 0.3  0.50   29.4
       0.4 0.0 0.0 0.4  0.75   29.4
       0.5 0.0 0.0 0.5  1.00   29.4
       0.6 0.0 0.0 0.6  1.00   29.4
       0.7 0.0 0.0 0.7  1.00   29.4
       0.8 0.0 0.0 0.8  1.00   29.4
       0.9 0.0 0.0 0.9  1.00   29.4
ngc0   0.0 0.0 0.0 0.0  1.00   29.4
       0.1 0.0 0.0 0.1  0.55   29.4
       0.2 0.0 0.0 0.2  1.00   29.4
       0.3 0.0 0.0 0.3  1.00   29.4
       0.4 0.0 0.0 0.4  1.00   29.4
       0.5 0.0 0.0 0.5  1.00   29.4
       0.6 0.0 0.0 0.6  1.00   29.4
       0.7 0.0 0.0 0.7  1.00   29.4
       0.8 0.0 0.0 0.8  1.00   29.4
       0.9 0.0 0.0 0.9  1.00   29.4
nri    0.0 0.0 0.0 0.0  0.85   29.4
       0.1 0.0 0.0 0.1  0.35   29.4
       0.2 0.0 0.0 0.2  0.40   29.4
       0.3 0.0 0.0 0.3  0.40   29.4
       0.4 0.0 0.0 0.4  0.40   29.4
       0.5 0.0 0.0 0.5  0.40   29.4
       0.6 0.0 0.0 0.6  0.40   29.4
       0.7 0.0 0.0 0.7  0.40   29.4
       0.8 0.0 0.0 0.8  0.45   29.4
       0.9 0.0 0.0 0.9  0.40   29.4